# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [4]:
from matplotlib import pyplot as plt
%pylab inline
plt.style.use("bmh")

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [5]:
import numpy as np
import pandas as pd

In [7]:
STUDENT = "Amir Alikulov and Ruslan Shuvalov "
ASSIGNMENT = 4
TEST = False

In [8]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 21

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [ ]:
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    df = df.copy()
    df.loc[df['Name'].str.contains('Julia Florence Siegel'), 'Age'] = 25
    return df

In [ ]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of male passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [4]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    required_columns = ['Age', 'Pclass', 'Sex', 'Embarked']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    filtered_df = df.loc[(df.Age >= 20) & (df.Age <= 30) & (df.Pclass == 3) & (df.Sex == 'male')]
    return filtered_df.Embarked.value_counts()

In [6]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

NameError: name 'TEST' is not defined

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [7]:
def fix_age(df):
    """Fix missing age values."""
    data = df.copy()

    mean_age = data.Age.mean()
    data.Age = data.Age.fillna(mean_age)

    print("Original missing ages:", df['Age'].isna().sum())
    print("New missing ages:", data['Age'].isna().sum())

    return data

In [8]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

NameError: name 'TEST' is not defined

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [9]:
def get_nursemaid(df):
    required_cols = ['Age', 'SibSp', 'Parch', 'Name']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"Missing required columns. Need: {required_cols}")

    data = df.copy()
    child = data.loc[(data.Age < 10) & (data.SibSp == 0) & (data.Parch == 0)]
    return tuple(child.iloc[0].Name.split(','))


In [10]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

NameError: name 'TEST' is not defined

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [ ]:
def get_port(df):
    """Get port with the most children embarked."""

    required_cols = ['Age', 'Embarked']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"Missing required columns. Need: {required_cols}")

    data = df.copy()
    child = data.loc[(data.Age < 10)]
    port = (child.groupby(data.Embarked).size() / data.groupby(data.Embarked).size()).idxmax()
    return port

In [ ]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [ ]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""

    required_cols = ['Ticket']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"Missing required columns. Need: {required_cols}")

    data = df.copy()

    ticket_size = data.groupby('Ticket').size()
    average_passengers_per_ticket = ticket_size.mean()
    max_passengers_per_ticket = ticket_size.max()

    return average_passengers_per_ticket, max_passengers_per_ticket


In [ ]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

The column `Fare` shows the Fare for the entire ticket, not price of one particular passenger on that ticket.
For each individual ticket, you should calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [ ]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    data = df.copy()
    fare_per_person_per_ticket = data.groupby(data.Ticket).Fare.first() / data.groupby(data.Ticket).size()
    data['Fare_per_person'] = data['Ticket'].map(fare_per_person_per_ticket)

    return data.groupby(['Pclass'])['Fare_per_person'].mean()


In [ ]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [ ]:
def fix_age_groupped(df):
    """Fill missing age values."""
    data = df.copy()
    data.groupby(['Pclass', 'Sex'])['Age'].mean()
    average_age_per_class_per_sex = data.groupby(['Pclass', 'Sex'])['Age'].mean()
    data['Age'] = data.apply(lambda x: average_age_per_class_per_sex.loc[x['Pclass'], x['Sex']] if pd.isnull(x['Age']) else x['Age'], axis=1)
    return data

In [ ]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [ ]:
def find_couples(df):
    """Your code here."""
    pass

In [ ]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

### 10. Lonely Passengers (2 points).

Extract # of passengers per class who were either traveling alone (no siblings/spouses/parents/children and also bought a ticket for one) or they have a ticket number beginning with "P", **but not both**.

Note that passenger traveling alone who has a ticket "PC 1234" **should NOT be counted**, but a passenger traveling alone with a ticket "AC 11" should be counted.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**.\
Output must be `Series` with three elements indexed by values from `Pclass` column.

In [ ]:
def lonely_or_p(df):
    """Your code here."""
    pass

In [ ]:
PROBLEM_ID = 10

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, lonely_or_p)

### 11. Family Tickets (3 points).

Find for each class a proportion of family tickets - tickets where all passengers have the same last name.\
Note that by that definition, even a 1-passenger ticket is considered a family ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column, with values between 0 and 1, containing the proportion of family tickets for that class.

In [ ]:
def family_tickets(df):
    """Your code here."""
    pass

In [ ]:
PROBLEM_ID = 11

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, family_tickets)

# Your grade

In [ ]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")